# Training and Evaluation of GNNs and LLMs
In this notebook, we train the models on the [MovieLens Dataset](https://movielens.org/) after the Pytorch Geometrics Tutorial on [Link Prediction](https://colab.research.google.com/drive/1xpzn1Nvai1ygd_P5Yambc_oe4VBPK_ZT?usp=sharing#scrollTo=vit8xKCiXAue).

First we import all of our dependencies.

The **GraphRepresentationGenerator** manages and trains a GNN model. Its most important interfaces include
**the constructor**, which defines the GNN architecture and loads the pre-trained GNN model if it is already on the hard disk,
**the training method**, which initializes the training on the GNN model and
**the get_embedding methods**, which represent the inference interface to the GNN model and return the corresponding embeddings in the dimension defined in the constructor for given user movie node pairs.

**The MovieLensLoader** loads and manages the data sets. The most important tasks include **saving and (re)loading and transforming** the data sets.

**PromptEncoderOnlyClassifier** and **VanillaEncoderOnlyClassifier** each manage a **prompt (model) LLM** and a **vanilla (model) LLM**. An EncoderOnlyClassifier (ClassifierBase) provides interfaces for training and testing an LLM model.
PromptEncoder and VanillaEncoder differ from their DataCollectors. DataCollectors change the behavior of the models during training and testing and allow data points to be created at runtime. With the help of these collators, we **create non-existent edges on the fly**.

In [1]:
from graph_representation_generator.graph_representation_generator import (
    GraphRepresentationGenerator,
)
from dataset_manager.movie_lens_manager import MovieLensManager
from dataset_manager.kg_manager import ROOT
from llm_manager.graph_prompter_hf.classifier import GraphPrompterHF

In [2]:
MODEL_NAME = "google/bert_uncased_L-2_H-128_A-2"
EPOCHS = 1
BATCH_SIZE_LLM = 256

We define in advance which **Knowledge Graph Embedding Dimension (KGE_DIMENSION)** the GNN encoder has. We want to determine from which output dimension the GNN encoder can produce embeddings that lead to a significant increase in performance *without exceeding the context length of the LLMs*. In the original tutorial, the KGE_DIMENSION was $64$.

In [3]:
kg_manager = MovieLensManager()

llm_df = kg_manager.llm_df.merge(kg_manager.target_df[["id", "prompt_feature_title", "prompt_feature_genres"]].rename(columns={"id": "target_id"}), on = "target_id")
llm_df

First we load the MovieLensLoader, which downloads the Movie Lens dataset (https://files.grouplens.org/datasets/movielens/ml-32m.zip) and prepares it to be used on GNN and LLM. We also pass the embedding dimensions that we will assume we are training with. First time takes approx. 30 sec.

In [4]:
kg_manager.data

HeteroData(
  source={ node_id=[200948] },
  target={
    node_id=[87585],
    x=[87585, 20],
  },
  (source, edge, target)={ edge_index=[2, 32000204] },
  (target, rev_edge, source)={ edge_index=[2, 32000204] }
)

Next, we initialize the GNN trainers (possible on Cuda), one for each KGE_DIMENSION.
A GNN trainer manages a model and each model consists of an **encoder and classifier** part.

**The encoder** is a parameterized *Grap Convolutional Network (GCN)* with a *2-layer GNN computation graph* and a single *ReLU* activation function in between.

**The classifier** applies the dot-product between source and destination kges to derive edge-level predictions.

In [5]:
graph_representation_generator_graph_prompter_hf = GraphRepresentationGenerator(
    kg_manager.data.to("cuda"),
    kg_manager.gnn_train_data.to("cuda"),
    kg_manager.gnn_val_data.to("cuda"),
    kg_manager.gnn_test_data.to("cuda"),
    hidden_channels=128,
    kge_dimension=128,
    force_recompute=False,
    device="cuda",
)

loading pretrained model
Device: 'cuda'


Next we produce the KGEs for every edge in the dataset. These embeddings can then be used for the LLM on the link-prediction task.

Next we initialize the vanilla encoder only classifier. This classifier does only use the NLP part of the prompt (no KGE) for predicting if the given link exists.

In [6]:
graph_prompter_hf_bert_classifier = GraphPrompterHF(
    kge_manager=kg_manager,
    get_embeddings_cb=graph_representation_generator_graph_prompter_hf.get_embeddings,
    model_name=MODEL_NAME,
    vanilla_model_path=f"{ROOT}/llm/vanilla/training/best",
    # gnn_parameters=list(
    #     graph_representation_generator_graph_prompter_hf.model.parameters()
    # ),
)

device cuda


Some weights of GraphPrompterHFBertForSequenceClassification were not initialized from the model checkpoint at ./data/llm/vanilla/training/best and are newly initialized because the shapes did not match:
- bert.embeddings.token_type_embeddings.weight: found shape torch.Size([3, 128]) in the checkpoint and torch.Size([5, 128]) in the model instantiated
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


5
5
5


In [7]:
dataset_embedding = kg_manager.generate_graph_prompter_hf_embedding_dataset(
    graph_prompter_hf_bert_classifier.tokenizer.sep_token,
    graph_prompter_hf_bert_classifier.tokenizer.pad_token,
    graph_prompter_hf_bert_classifier.tokenize_function,
)

Loading dataset from disk:   0%|          | 0/63 [00:00<?, ?it/s]

Loading dataset from disk:   0%|          | 0/23 [00:00<?, ?it/s]

Loading dataset from disk:   0%|          | 0/23 [00:00<?, ?it/s]

import torch

gnn_model = graph_representation_generator_graph_prompter_hf.model
trainer = graph_prompter_hf_bert_classifier._get_trainer(
    dataset_embedding, epochs=EPOCHS, batch_size=BATCH_SIZE_LLM
)
optimizer = torch.optim.SGD(
    list(trainer.model.parameters()) + list(gnn_model.parameters()),
    lr=0.001,
    momentum=0.9,
)


def inspect_gradients(trainer, batch):
    optimizer.zero_grad()  # Clear previous gradients
    trainer.model.eval()  # Set model to eval mode (no dropout, etc.)
    outputs = trainer.model(**batch)
    loss = outputs.loss
    loss.backward()  # Run backward to compute gradients
    optimizer.step()
    for name, param in gnn_model.named_parameters():
        if param.requires_grad and param.grad is not None:
            print(f"Gradient for gnn {name}: {param.grad}")

    # for name, param in trainer.model.named_parameters():
    #     if param.requires_grad and param.grad is not None:
    #         print(f"Gradient for LLM{name}: {(param.grad > 0).any()}")


# Example of grabbing a batch and inspecting gradients
for i in range(1):
    sample_batch = next(iter(trainer.get_train_dataloader()))

    inspect_gradients(trainer, sample_batch)

tens_1 = graph_representation_generator_graph_prompter_hf.model.to(
    device="cpu"
).state_dict()

tens_2 = torch.load("./data/gnn/backup/model_128.pth")
same = True
for key in tens_1.keys():
    if isinstance(tens_1[key], torch.Tensor):
        if not (tens_1[key] == tens_2[key]).all():
            same = False
print(same)

In [8]:
graph_prompter_hf_bert_classifier.train_model_on_data(
    dataset_embedding,
    epochs=EPOCHS,
    batch_size=BATCH_SIZE_LLM,
)
graph_representation_generator_graph_prompter_hf.save_model()

  0%|          | 0/70001 [00:00<?, ?it/s]

c:\Users\MARS\.conda\envs\hauptprojekt\Lib\site-packages\transformers\models\bert\modeling_bert.py:440: UserWarning: 1Torch was not compiled with flash attention. (Triggered internally at ..\aten\src\ATen\native\transformers\cuda\sdp_utils.cpp:455.)
  attn_output = torch.nn.functional.scaled_dot_product_attention(


{'loss': 0.433, 'grad_norm': 7.652825355529785, 'learning_rate': 1.0000000000000002e-06, 'epoch': 0.0}
{'loss': 0.4166, 'grad_norm': 5.707124710083008, 'learning_rate': 2.0000000000000003e-06, 'epoch': 0.0}
{'loss': 0.407, 'grad_norm': 6.322635650634766, 'learning_rate': 3e-06, 'epoch': 0.0}
{'loss': 0.346, 'grad_norm': 3.9728102684020996, 'learning_rate': 4.000000000000001e-06, 'epoch': 0.0}
{'loss': 0.3405, 'grad_norm': 3.7028260231018066, 'learning_rate': 5e-06, 'epoch': 0.0}
{'loss': 0.3469, 'grad_norm': 2.8664913177490234, 'learning_rate': 6e-06, 'epoch': 0.0}
{'loss': 0.319, 'grad_norm': 2.752406597137451, 'learning_rate': 7.000000000000001e-06, 'epoch': 0.0}
{'loss': 0.3194, 'grad_norm': 1.8753429651260376, 'learning_rate': 8.000000000000001e-06, 'epoch': 0.0}
{'loss': 0.313, 'grad_norm': 2.2589833736419678, 'learning_rate': 9e-06, 'epoch': 0.0}
{'loss': 0.2825, 'grad_norm': 1.926758050918579, 'learning_rate': 1e-05, 'epoch': 0.0}
{'loss': 0.2955, 'grad_norm': 2.086960792541504,

TypeError: Object of type HeteroData is not JSON serializable

In [15]:
import torch

tens_1 = graph_representation_generator_graph_prompter_hf.model.to(
    device="cpu"
).state_dict()

tens_2 = torch.load("./data/gnn/backup/model_128.pth")
same = True
for key in tens_1.keys():
    if isinstance(tens_1[key], torch.Tensor):
        if not (tens_1[key] == tens_2[key]).all():
            same = False
print(same)

target_lin.weight is same
target_lin.bias is same
source_emb.weight is same
target_emb.weight is same
gnn.conv1.source__edge__target.lin_l.weight is same
gnn.conv1.source__edge__target.lin_l.bias is same
gnn.conv1.source__edge__target.lin_r.weight is same
gnn.conv1.target__rev_edge__source.lin_l.weight is same
gnn.conv1.target__rev_edge__source.lin_l.bias is same
gnn.conv1.target__rev_edge__source.lin_r.weight is same
gnn.conv2.source__edge__target.lin_l.weight is same
gnn.conv2.source__edge__target.lin_l.bias is same
gnn.conv2.source__edge__target.lin_r.weight is same
gnn.conv2.target__rev_edge__source.lin_l.weight is same
gnn.conv2.target__rev_edge__source.lin_l.bias is same
gnn.conv2.target__rev_edge__source.lin_r.weight is same
